Classify 1-4 Scale Separately

In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Reshape, Dense, GlobalAveragePooling2D
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, confusion_matrix
from keras import models, optimizers, Sequential, regularizers, layers
from keras.models import load_model, model_from_json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.convolutional import Conv2D, MaxPooling2D
import itertools

np.random.seed(123)

Using TensorFlow backend.


In [4]:
pwd

'/Users/justinmitchell/Desktop/python_files/data_science_practice/diabetic_retinopathy'

In [7]:
directory = 'train_images/'

data_all = ImageDataGenerator(rescale=1./255).flow_from_directory( 
        directory,
        class_mode='sparse',
        target_size=(224, 224), 
        batch_size = 3662, 
        seed = 123)

Found 1857 images belonging to 4 classes.


In [8]:
data_all.class_indices

{'1': 0, '2': 1, '3': 2, '4': 3}

In [ ]:
images, labels = next(data_all)

In [5]:
X_model, X_test, y_model, y_test = train_test_split(images, labels, test_size=0.20, random_state=123, stratify = labels)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X_model, y_model, test_size=0.20, random_state=123, stratify = y_model)

In [9]:
def baseline_model():
    model = Sequential()
    model.add(layers.Conv2D(224, (3, 3), activation='relu', input_shape=(224,224,3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(112, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(56, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(300, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(150, activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
    return model

In [10]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [ ]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))